In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv


In [4]:
os.listdir('../input/telco-customer-churn')

['WA_Fn-UseC_-Telco-Customer-Churn.csv']

In [24]:
telco=pd.read_csv('../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [16]:
telco.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [15]:
telco.shape

(7043, 20)

In [13]:
telco.isnull().sum().sum()

0

In [25]:
telco.drop(['customerID'],axis=1,inplace=True)

In [18]:
telco['Contract']

0       Month-to-month
1             One year
2       Month-to-month
3             One year
4       Month-to-month
             ...      
7038          One year
7039          One year
7040    Month-to-month
7041    Month-to-month
7042          Two year
Name: Contract, Length: 7043, dtype: object

In [20]:
list(telco.columns)

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [26]:
telco['Churn']=telco['Churn'].apply(lambda x : 1 if x=='Yes' else 0)

In [28]:
telco.groupby('gender')['Churn'].sum()

gender
Female    939
Male      930
Name: Churn, dtype: int64

In [35]:
telco_cat_col=[i for i in telco.columns if telco[i].dtypes==object]

In [37]:
telco_nu_col=telco.columns.difference(telco_cat_col)

In [42]:
telco_cat=telco.loc[:,telco_cat_col]

In [47]:
telco_nu=telco.loc[:,telco_nu_col]

In [44]:
Total_charge=telco_cat[['TotalCharges']]

In [46]:
telco_cat.drop(['TotalCharges'],axis=1,inplace=True)

In [48]:
telco_nu=pd.concat([telco_nu,Total_charge],axis=1)

In [49]:
telco_nu.head()

,Churn,MonthlyCharges,SeniorCitizen,tenure,TotalCharges
0,0,29.85,0,1,29.85
1,0,56.95,0,34,1889.5
2,1,53.85,0,2,108.15
3,0,42.30,0,45,1840.75
4,1,70.70,0,2,151.65


In [63]:
num_cat=[]

for i in telco_cat.columns:
    k=telco_cat[i].unique()
    num_cat.append([i,k,len(k)])

In [64]:
pd.DataFrame(num_cat)

,0,1,2
0,gender,"[Female, Male]",2
1,Partner,"[Yes, No]",2
2,Dependents,"[No, Yes]",2
3,PhoneService,"[No, Yes]",2
4,MultipleLines,"[No phone service, No, Yes]",3
5,InternetService,"[DSL, Fiber optic, No]",3
6,OnlineSecurity,"[No, Yes, No internet service]",3
7,OnlineBackup,"[Yes, No, No internet service]",3
8,DeviceProtection,"[No, Yes, No internet service]",3
9,TechSupport,"[No, Yes, No internet service]",3


In [68]:
telco_dummy=pd.get_dummies(telco_cat,drop_first=True)

In [70]:
telco_dummy.shape

(7043, 26)

In [74]:
num_nu=[]

for i in telco_nu.columns:
    k=telco_nu[i].unique()
    num_nu.append([i,k,len(k)])

In [77]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='whitegrid')

In [97]:
telco_nu['TotalCharges']=telco_nu['TotalCharges'].replace(" ",0)

In [102]:
telco_nu['TotalCharges'].dtypes

dtype('float64')

In [107]:
from sklearn.preprocessing import RobustScaler

In [108]:
robust=RobustScaler().fit_transform([telco_nu.MonthlyCharges,telco_nu.tenure,telco_nu.TotalCharges])


In [113]:
robust

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-2.        , -0.02473727, -0.97691945, ..., -0.11089581,
        -0.46530073, -0.01169875],
       [ 0.        ,  1.97526273,  1.02308055, ...,  1.88910419,
         1.53469927,  1.98830125]])

In [115]:
telco_nu.drop(['MonthlyCharges','tenure','TotalCharges'],axis=1,inplace=True)

In [119]:
telco_nu_final=pd.concat([telco_nu,telco_nu_sub],axis=1)

In [120]:
telco_nu_final

,Churn,SeniorCitizen,0,1,2
0,0,0,0.000000,-2.000000,0.000000
1,0,0,0.000000,-0.024737,1.975263
2,1,0,0.000000,-0.976919,1.023081
3,0,0,-0.003003,0.000000,1.996997
4,1,0,0.000000,-0.918142,1.081858
...,...,...,...,...,...
7038,0,0,0.000000,-0.061836,1.938164
7039,0,0,0.000000,-0.008559,1.991441
7040,0,0,0.000000,-0.110896,1.889104
7041,1,1,0.000000,-0.465301,1.534699


In [131]:
telco_nu_final=telco_nu_final.rename(columns={0:'MonthlyCharges',1:'tenure',2:'TotalCharges'})

In [132]:
telco_nu_final.columns

Index(['Churn', 'SeniorCitizen', 'MonthlyCharges', 'tenure', 'TotalCharges'], dtype='object')

In [133]:
telco_nu_final

,Churn,SeniorCitizen,MonthlyCharges,tenure,TotalCharges
0,0,0,0.000000,-2.000000,0.000000
1,0,0,0.000000,-0.024737,1.975263
2,1,0,0.000000,-0.976919,1.023081
3,0,0,-0.003003,0.000000,1.996997
4,1,0,0.000000,-0.918142,1.081858
...,...,...,...,...,...
7038,0,0,0.000000,-0.061836,1.938164
7039,0,0,0.000000,-0.008559,1.991441
7040,0,0,0.000000,-0.110896,1.889104
7041,1,1,0.000000,-0.465301,1.534699


In [138]:
telco_final=pd.concat([telco_nu_final,telco_dummy],axis=1)

In [140]:
telco_final.isnull().sum()

Churn                                    0
SeniorCitizen                            0
MonthlyCharges                           0
tenure                                   0
TotalCharges                             0
gender_Male                              0
Partner_Yes                              0
Dependents_Yes                           0
PhoneService_Yes                         0
MultipleLines_No phone service           0
MultipleLines_Yes                        0
InternetService_Fiber optic              0
InternetService_No                       0
OnlineSecurity_No internet service       0
OnlineSecurity_Yes                       0
OnlineBackup_No internet service         0
OnlineBackup_Yes                         0
DeviceProtection_No internet service     0
DeviceProtection_Yes                     0
TechSupport_No internet service          0
TechSupport_Yes                          0
StreamingTV_No internet service          0
StreamingTV_Yes                          0
StreamingMo

In [141]:
import lightgbm as lgb

In [143]:
from sklearn.model_selection import train_test_split

In [144]:
train=telco_final.drop(['Churn'],axis=1)
label=telco_final['Churn']

In [145]:
x_train,x_test,y_train,y_test=train_test_split(train,label,test_size=0.2,random_state=2020)

# SMOTE 사용
- when target data is imbalance, we can apply smote method to this case

In [147]:
telco_final['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [148]:
from imblearn.over_sampling import SMOTE
oversampler=SMOTE(random_state=2020)
smote_train,smote_target=oversampler.fit_sample(x_train,y_train)

Using TensorFlow backend.


# LightGBM
-  I mention about importance of parameter following code, where each paramas are ranked depending on the nember of stars.

In [296]:
categorical=[0,1]

def lgb_model_fit(params,sm_train,sm_target,x_test,y_test,
                 objective='binary',metrics='auc',feval=None,verbose_eval=10,
                 early_stopping_rounds=30,num_boost_round=10000,
                 categorical_feature=None):
    lgb_params={
        'boosting_type':'gbdt',
        'objective':objective,
        'metric':metrics,
        'learning_rate':0.01,
        'num_leaves':31,
        'max_depth':-1,
        'min_child_samples':20,
        'max_bin':255,
        'subsample_freq':0,
        'colsample_bytree':0.3,
        'min_child_weight':2,
        'subsample_for_bin':2000,
        'min_split_gain':4,
        'reg_alpha':0,
        'reg_lambda':0,
        'scale_pos_weight':1.1,
        'nthread':4
    }
    
    lgb_params.update(params)
    print('preparing validation datasets')
    
    xgtrain=lgb.Dataset(sm_train,label=sm_target,
                       feature_name='auto')
    xgvalid=lgb.Dataset(x_test,label=y_test,
                       feature_name='auto')
    
    evals_results={}
    
    bst1=lgb.train(lgb_params,
                  xgtrain,
                  valid_sets=[xgtrain,xgvalid],
                  valid_names=['train','valid'],
                  evals_result=evals_results,
                  num_boost_round=num_boost_round,
                  early_stopping_rounds=early_stopping_rounds,
                  verbose_eval=10,
                  feval=feval)
    n_estimators=bst1.best_iteration
    print('\n Model Report')
    print('\n n_estimators:',n_estimators)
    return bst1

In [297]:
params={
    'boosting_type':'gbdt', # ***
    'learning_rate':0.1, # ***
    'max_depth':-1, # ***
    'num_leaves':6, # * (2^max_depth) -1
    'min_child_samples':10, # *
    'max_bin':15,# **
    'subsample':0.7, # ** bagging_fraction
    'subsample_freq':0, # * bagging_freq
    'colsample_bytree':0.8, # ** feature_fraction
    'min_child_weight':3, # **
    'scale_pos_weight':1, # *
    'reg_lambda':0.6, # **
    'reg_alpha':0.5 # **
}

bst=lgb_model_fit(params,
                 sm_train=smote_train,
                 sm_target=smote_target,
                 x_test=x_test,
                 y_test=y_test,
                 objective='binary',
                 metrics='auc',
                 early_stopping_rounds=100, #**
                 verbose_eval=True,
                 num_boost_round=100000 #***
                )

preparing validation datasets
Training until validation scores don't improve for 100 rounds
[10]	train's auc: 0.858929	valid's auc: 0.831261
[20]	train's auc: 0.866611	valid's auc: 0.838139
[30]	train's auc: 0.871357	valid's auc: 0.842543
[40]	train's auc: 0.875029	valid's auc: 0.844544
[50]	train's auc: 0.876649	valid's auc: 0.845255
[60]	train's auc: 0.877045	valid's auc: 0.844881
[70]	train's auc: 0.877045	valid's auc: 0.844881
[80]	train's auc: 0.877045	valid's auc: 0.844881
[90]	train's auc: 0.877045	valid's auc: 0.844881
[100]	train's auc: 0.877045	valid's auc: 0.844881
[110]	train's auc: 0.877045	valid's auc: 0.844881
[120]	train's auc: 0.877045	valid's auc: 0.844881
[130]	train's auc: 0.877045	valid's auc: 0.844881
[140]	train's auc: 0.877045	valid's auc: 0.844881
[150]	train's auc: 0.877045	valid's auc: 0.844881
Early stopping, best iteration is:
[50]	train's auc: 0.876649	valid's auc: 0.845255

 Model Report

 n_estimators: 50
